In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style('darkgrid')

In [3]:
import sklearn as sk
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

In [4]:
df = pd.read_csv('analytical_base_table.csv')

In [5]:
df.head(10)

,Product_Info_1,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,BMI,Employment_Info_1,Employment_Info_2,...,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48,Response,Product_Info_2_A8,Product_Info_2_D1,Product_Info_2_D2,Product_Info_2_D3,Product_Info_2_D4,Product_Info_2_spares_data
0,1,10,0.076923,2,1,1,0.641791,0.323008,0.028,12,...,0,0,0,8,0,0,0,1,0,0
1,1,26,0.076923,2,3,1,0.059701,0.272288,0.000,1,...,0,0,0,4,0,0,0,0,0,1
2,1,26,0.076923,2,3,1,0.029851,0.428780,0.030,9,...,0,0,0,8,0,0,0,0,0,1
3,1,10,0.487179,2,3,1,0.164179,0.352438,0.042,9,...,0,0,0,8,0,0,0,0,1,0
4,1,26,0.230769,2,3,1,0.417910,0.424046,0.027,9,...,0,0,0,8,0,0,1,0,0,0
5,1,26,0.230769,3,1,1,0.507463,0.364887,0.325,15,...,0,0,0,8,0,0,1,0,0,0
6,1,10,0.166194,2,3,1,0.373134,0.376587,0.110,1,...,0,0,0,8,1,0,0,0,0,0
7,1,26,0.076923,2,3,1,0.611940,0.571612,0.120,12,...,0,0,0,1,0,0,1,0,0,0
8,1,26,0.230769,2,3,1,0.522388,0.362643,0.165,9,...,0,0,0,8,0,0,0,1,0,0
9,1,21,0.076923,2,3,1,0.552239,0.587796,0.025,1,...,0,0,0,1,0,0,0,0,0,1


In [6]:
df.shape

(59381, 120)

Spliting the Dataframe into randomized subsets. to train our model

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
#creating seperate object for target variable
y = df.Response

#creating seperate object for the input features
X = df.drop('Response', axis=1)

In [9]:
X.shape, y.shape

((59381, 119), (59381,))

* Now we pass X and y to the train_test_split() function
    * you'll split X into X_train and X_test.
    * you'll split into y_train and y_test.

In [10]:
#tewst_size=0.2 set aside 20% of our observation for our test set.
#split X and y into train and test sets
#it returns a tuple with 4 elements
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=1234)

To comfirm we the right kind of observation in each subset

In [11]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

47504 11877 47504 11877


* Next we fit X_train features values and y_train values.
* We will use the trained models to predict X_test and evaluate the predictions against y_test


# Cross validation : getting a reliable estimate of a model performance using only our training data.
    we use the 10 fold cross validation

# Preprocessing and pipeline
 * we standardize all our features to the same scale making the feature distribution center around zero with unit variance

In [12]:
X_train.describe()

,Product_Info_1,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,BMI,Employment_Info_1,Employment_Info_2,...,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48,Product_Info_2_A8,Product_Info_2_D1,Product_Info_2_D2,Product_Info_2_D3,Product_Info_2_D4,Product_Info_2_spares_data
count,47504.000000,47504.000000,47504.000000,47504.000000,47504.000000,47504.000000,47504.000000,47504.000000,47504.000000,47504.000000,...,47504.000000,47504.000000,47504.000000,47504.000000,47504.000000,47504.000000,47504.000000,47504.000000,47504.000000,47504.000000
mean,1.026061,24.418975,0.329441,2.007263,2.673249,1.043365,0.405801,0.469322,0.077676,8.644809,...,0.013894,0.008736,0.019893,0.054859,0.115190,0.109528,0.106496,0.240085,0.182090,0.246611
std,0.159318,5.064608,0.283027,0.084911,0.739424,0.291219,0.197038,0.121784,0.082512,4.242869,...,0.117051,0.093059,0.139634,0.227706,0.319255,0.312303,0.308475,0.427139,0.385923,0.431043
min,1.000000,1.000000,0.000000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,26.000000,0.076923,2.000000,3.000000,1.000000,0.238806,0.385731,0.035000,9.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,26.000000,0.230769,2.000000,3.000000,1.000000,0.402985,0.451312,0.060000,9.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,26.000000,0.487179,2.000000,3.000000,1.000000,0.567164,0.532772,0.099557,9.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.000000,38.000000,1.000000,3.000000,3.000000,3.000000,1.000000,1.000000,1.000000,37.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
#feature scaling


In [16]:
# addressing rounding up issue by setting a pandas display option
# this option will show all values in pandas as numbers with 3 decimals
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [17]:
X_train_new.describe()

,Product_Info_1,Product_Info_3,Product_Info_4,Product_Info_5,Product_Info_6,Product_Info_7,Ins_Age,BMI,Employment_Info_1,Employment_Info_2,...,Medical_Keyword_45,Medical_Keyword_46,Medical_Keyword_47,Medical_Keyword_48,Product_Info_2_A8,Product_Info_2_D1,Product_Info_2_D2,Product_Info_2_D3,Product_Info_2_D4,Product_Info_2_spares_data
count,47504.000,47504.000,47504.000,47504.000,47504.000,47504.000,47504.000,47504.000,47504.000,47504.000,...,47504.000,47504.000,47504.000,47504.000,47504.000,47504.000,47504.000,47504.000,47504.000,47504.000
mean,0.000,0.000,-0.000,0.000,-0.000,0.000,-0.000,-0.000,0.000,-0.000,...,0.000,0.000,0.000,0.000,-0.000,-0.000,0.000,-0.000,0.000,-0.000
std,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
min,-0.164,-4.624,-1.164,-0.086,-2.263,-0.149,-2.060,-3.854,-0.941,-1.802,...,-0.119,-0.094,-0.142,-0.241,-0.361,-0.351,-0.345,-0.562,-0.472,-0.572
25%,-0.164,0.312,-0.892,-0.086,0.442,-0.149,-0.848,-0.686,-0.517,0.084,...,-0.119,-0.094,-0.142,-0.241,-0.361,-0.351,-0.345,-0.562,-0.472,-0.572
50%,-0.164,0.312,-0.349,-0.086,0.442,-0.149,-0.014,-0.148,-0.214,0.084,...,-0.119,-0.094,-0.142,-0.241,-0.361,-0.351,-0.345,-0.562,-0.472,-0.572
75%,-0.164,0.312,0.557,-0.086,0.442,-0.149,0.819,0.521,0.265,0.084,...,-0.119,-0.094,-0.142,-0.241,-0.361,-0.351,-0.345,-0.562,-0.472,-0.572
max,6.113,2.682,2.369,11.691,0.442,6.719,3.016,4.358,11.178,6.683,...,8.425,10.652,7.019,4.151,2.771,2.851,2.897,1.779,2.119,1.748


# training our model

In [ ]:

LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_tr, y_tr)  
LR.predict(X_test)  
round(LR.score(X_test,y_test), 4)

RF = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=0).fit(X_tr, y_tr)  
RF.predict(X_test)  
round(RF.score(X_test, y_test), 4)


SVM = svm.SVC(decision_function_shape="ovo").fit(X_train, y_train)  
SVM.predict(X_test)  
round(SVM.score(X_test, y_test), 4)